In [1]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0, base_url='https://api.deepseek.com', model='deepseek-chat')

graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()
runnable

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<__root__>(recurse=True, writes=[ChannelWriteEntry(channel='__root__', value=<object object at 0x7ff954c5b570>, skip_none=True, mapper=None)]), ChannelWrite<start:oracle>(recurse=True, writes=[ChannelWriteEntry(channel='start:oracle', value='__start__', skip_none=False, mapper=None)])]), 'oracle': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['start:oracle'], writers=[ChannelWrite<oracle,__root__>(recurse=True, writes=[ChannelWriteEntry(channel='oracle', value='oracle', skip_none=False, mapper=None), ChannelWriteEntry(channel='__root__', value=<object object at 0x7ff954c5b570>, skip_none=True, mapper=None)])])}, channels={'__root__': <langgraph.channels.binop.BinaryOperatorAggregate object at 0x7ff93f5a31c0>, '__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7ff93f7f4af0>, 'oracle': <la

In [3]:
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='f2a6a7f2-b76a-4016-b9e7-84bd3b45d81f'),
 AIMessage(content=' 1 + 1 equals 2.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 15, 'total_tokens': 24}, 'model_name': 'deepseek-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4cfe5c6e-97a5-427a-8dc3-6ab61fa94566-0')]